## Softmax

In [1]:
import numpy as np

In [19]:
vector = np.array([11231, 999, 123142])

In [28]:
vector = np.array([-10, 10, 24])

In [33]:
def softmax(vec):
    vec -= np.max(vec)
    exp = np.exp(vec)
    return exp / np.sum(exp)

In [32]:
softmax(vector)

array([1.71390701e-15, 8.31528028e-07, 9.99999168e-01])

## Assignment-04 基于维基百科的词向量构建

在本章，你将使用Gensim和维基百科获得你的第一批词向量，并且感受词向量的基本过程。

![](https://www.kaggleusercontent.com/kf/1018109/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..JNNggcCCDcYEypvp7ZDwOA.cM9CuDpuCKo0K_ZkMFLAUvhfip0P6SRZ4LddwgTtgwz8pQy1dZeGVJWi6u81KSpAFNSi7YximVVJbPw8xsFySdWlqoUwvSER-LLIRfmlpsCvtDt90NaLYT2FHlwl0tfF-1MKtiFsWlGQ8LGo40hL3ccBSwMZy214kGJf9bNkW_g.kZbF5sgN5qha3zhjilfSDg/__results___files/__results___9_0.png)

## Step-01: Download Wikipedia Chinese Corpus: https://dumps.wikimedia.org/zhwiki/20190720/

第一步：使用维基百科下载中文语料库

``downloaded file: zhwiki-20190720-pages-articles-multistream.xml.bz2``

## Step-02: Using https://github.com/attardi/wikiextractor to extract the wikipedia corpus

第二步：使用python wikipedia extractor抽取维基百科的内容

```
bzip2 -d zhwiki-20190720-pages-articles-multistream.xml.bz2 # decompress
./wikiextractor/WikiExtractor.py zhwiki-20190720-pages-articles-multistream.xml # extract
./text # extracted file location
```

## Step-03: Using gensim get word vectors: 

Reference: 

+ https://radimrehurek.com/gensim/models/word2vec.html
+ https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

第三步：参考Gensim的文档和Kaggle的参考文档，获得词向量。 注意，你要使用Jieba分词把维基百科的内容切分成一个一个单词，然后存进新的文件中。然后，你需要用Gensim的LineSentence这个类进行文件的读取。

在训练成词向量Model.

In [8]:
import sys
!{sys.executable} -m pip install jieba beautifulsoup4

In [49]:
import json
import jieba
import os
import re
from bs4 import BeautifulSoup
corp_file = os.path.abspath("./wiki_processed.corp")
corps =  open(corp_file, 'a+')


def clean(string):
    return ''.join(re.findall(r'\w+', string))

def cut(string):
    return list(jieba.cut(string))

def preprocess_wiki_data(root_dir):
    dirs = os.listdir(root_dir)
    for d in dirs:
        temp = root_dir + '/' + d
        if os.path.isdir(temp):
            preprocess_wiki_data(temp)
        elif os.path.exists(temp):
            corps_str = ''
            with open(temp, 'r') as f:
               
                soup = BeautifulSoup(f)
                docs = soup.select('doc')
                for doc in docs:
                    content = str(doc.string)
                    corp = cut(clean(content))
                    corp_str = ' '.join(corp)
                    corps.write(corp_str + "\n")
        else:
            return
    print("Finished processing files under: {}".format(root_dir))
            
preprocess_wiki_data('./text')

Finished processing files under: ./text/AL
Finished processing files under: ./text/AM
Finished processing files under: ./text/AI
Finished processing files under: ./text/AD
Finished processing files under: ./text/AK
Finished processing files under: ./text/AA
Finished processing files under: ./text/AF
Finished processing files under: ./text/AN
Finished processing files under: ./text/AH
Finished processing files under: ./text/AE
Finished processing files under: ./text/AG
Finished processing files under: ./text/AJ
Finished processing files under: ./text/AC
Finished processing files under: ./text/AB
Finished processing files under: ./text


## Step-04: Using some words to test your preformance. 

In [3]:
import sys
!{sys.executable} -m pip install --upgrade gensim

Looking in indexes: https://pypi.douban.com/simple
    100% |████████████████████████████████| 24.7MB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 433kB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 436kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 174kB/s ta 0:00:011
    100% |████████████████████████████████| 5.6MB 1.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /Users/weiliang/Library/Caches/pip/wheels/d6/99/34/8ef455ebe1eac3de61bcf4793a3c1ef68431ca3ebeec804488
Successfully built smart-open


In [1]:
import os
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
model_file = "zh_word2vec.model"
if os.path.exists(model_file):
    model = Word2Vec.load(model_file)
else:
    sentences = LineSentence('./wiki_processed.corp')
    model = Word2Vec(sentences, size=100, window=5, min_count=100, workers=4)
    model.save(model_file)


In [2]:
model.wv.most_similar(['西瓜'])

[('韭菜', 0.8098064661026001),
 ('地瓜', 0.8060703873634338),
 ('芹菜', 0.8038861751556396),
 ('冬瓜', 0.8020275235176086),
 ('柿子', 0.7961002588272095),
 ('瓜果', 0.7946608662605286),
 ('茄子', 0.7857564091682434),
 ('油桐', 0.7835367321968079),
 ('蔥', 0.7817407846450806),
 ('白菜', 0.7795143127441406)]

In [3]:
model.wv.most_similar(['恶心'])

[('头痛', 0.9274026155471802),
 ('呕吐', 0.9161430597305298),
 ('头晕', 0.8861633539199829),
 ('呼吸困难', 0.8693729639053345),
 ('腹泻', 0.8650820255279541),
 ('心悸', 0.8102750778198242),
 ('眩晕', 0.8072993159294128),
 ('焦虑', 0.8028079867362976),
 ('狂躁', 0.8024279475212097),
 ('抑郁', 0.802391767501831)]

第四步，测试同义词，找几个单词。

## Step-05: Using visualization tools: https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

第五步：使用Kaggle给出的T-SEN进行词向量的可视化。

In [4]:
from sklearn.manifold import TSNE

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
tsne_plot(model) # This function takes too much time and never draw successfully on my local machine.

/Users/weiliang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
